In [1]:
import os, sys
import numpy as np
# path to access c++ files
installation_path = os.getenv("INSTALL_PATH")
sys.path.append(installation_path)

In [2]:
from cunqa import getQPUs

qpus  = getQPUs()

for q in qpus:
    print(f"QPU {q.id}, backend: {q.backend.name}, simulator: {q.backend.simulator}, version: {q.backend.version}.")


	info: Logger created.
	debug: File accessed correctly.
	debug: Object for QPU 0 created correctly.
	debug: Object for QPU 1 created correctly.
	debug: Object for QPU 2 created correctly.
	debug: Object for QPU 3 created correctly.
	debug: Object for QPU 4 created correctly.
	debug: Object for QPU 5 created correctly.
	debug: Object for QPU 6 created correctly.
	debug: Object for QPU 7 created correctly.
	debug: Object for QPU 8 created correctly.
	debug: Object for QPU 9 created correctly.
	debug: Object for QPU 10 created correctly.
	debug: Object for QPU 11 created correctly.
	debug: Object for QPU 12 created correctly.
	debug: Object for QPU 13 created correctly.
	debug: Object for QPU 14 created correctly.
	debug: Object for QPU 15 created correctly.
	debug: Object for QPU 16 created correctly.
	debug: Object for QPU 17 created correctly.
	debug: Object for QPU 18 created correctly.
	debug: Object for QPU 19 created correctly.
	debug: Object for QPU 20 created correctly.
	debug: O

# Paralelization for gradient-free optimizers: Differential Evolution

_Introduction and explanation_

We recover the variational circuit used before:

In [3]:
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter

def hardware_efficient_ansatz(num_qubits, num_layers):
    qc = QuantumCircuit(num_qubits)
    param_idx = 0
    for _ in range(num_layers):
        for qubit in range(num_qubits):
            phi = Parameter(f'phi_{param_idx}_{qubit}')
            lam = Parameter(f'lam_{param_idx}_{qubit}')
            qc.ry(phi, qubit)
            qc.rz(lam, qubit)
        param_idx += 1
        for qubit in range(num_qubits - 1):
            qc.cx(qubit, qubit + 1)
    qc.measure_all()
    return qc

In [4]:
def target_distribution(num_qubits):
    # Define a normal distribution over the states
    num_states = 2 ** num_qubits
    states = np.arange(num_states)
    mean = num_states / 2
    std_dev = num_states / 4
    target_probs = norm.pdf(states, mean, std_dev)
    target_probs /= target_probs.sum()  # Normalize to make it a valid probability distribution
    target_dist = {format(i, f'0{num_qubits}b'): target_probs[i] for i in range(num_states)}
    return target_dist

import pandas as pd
from scipy.stats import entropy, norm

def KL_divergence(counts, n_shots, target_dist):
    # Convert counts to probabilities
    pdf = pd.DataFrame.from_dict(counts, orient="index").reset_index()
    pdf.rename(columns={"index": "state", 0: "counts"}, inplace=True)
    pdf["probability"] = pdf["counts"] / n_shots
    
    # Create a dictionary for the obtained distribution
    obtained_dist = pdf.set_index("state")["probability"].to_dict()
    
    # Ensure all states are present in the obtained distribution
    for state in target_dist:
        if state not in obtained_dist:
            obtained_dist[state] = 0.0
    
    # Convert distributions to lists for KL divergence calculation
    target_probs = [target_dist[state] for state in sorted(target_dist)]
    obtained_probs = [obtained_dist[state] for state in sorted(obtained_dist)]
    
    # Calculate KL divergence
    kl_divergence = entropy(obtained_probs, target_probs)
    
    return kl_divergence
    

In [5]:
num_qubits = 6

num_layers = 3

n_shots = 100000

target_dist = target_distribution(num_qubits)

In [6]:
def cost_function(result):
    
    global target_dist
    
    counts = result.get_counts()
    
    return KL_divergence(counts, n_shots, target_dist)

In [7]:
ansatz = hardware_efficient_ansatz(num_qubits, num_layers)

num_parameters = ansatz.num_parameters; print(num_parameters)

initial_parameters = np.zeros(num_parameters)

36


In [8]:
init_qjobs = []
init_params = np.zeros(num_parameters)
for q in qpus:
    init_qjobs.append(q.run(ansatz.assign_parameters(init_params), transpile=False, shots=n_shots))

from cunqa import QJobMapper
mapper = QJobMapper(init_qjobs)

	debug: A QuantumCircuit was provided.
	debug: Translating to dict for AerSimulator...
	debug: QJob created.
	debug:  {"config":{"shots": 100000, "method": "statevector", "memory_slots": 6, "seed": 188}, "instructions":[{"name": "ry", "qubits": [0], "params": [0.0]}, {"name": "rz", "qubits": [0], "params": [0.0]}, {"name": "ry", "qubits": [1], "params": [0.0]}, {"name": "rz", "qubits": [1], "params": [0.0]}, {"name": "ry", "qubits": [2], "params": [0.0]}, {"name": "rz", "qubits": [2], "params": [0.0]}, {"name": "ry", "qubits": [3], "params": [0.0]}, {"name": "rz", "qubits": [3], "params": [0.0]}, {"name": "ry", "qubits": [4], "params": [0.0]}, {"name": "rz", "qubits": [4], "params": [0.0]}, {"name": "ry", "qubits": [5], "params": [0.0]}, {"name": "rz", "qubits": [5], "params": [0.0]}, {"name": "cx", "qubits": [0, 1], "params": []}, {"name": "cx", "qubits": [1, 2], "params": []}, {"name": "cx", "qubits": [2, 3], "params": []}, {"name": "cx", "qubits": [3, 4], "params": []}, {"name": "cx

	debug: Circuit was sent.
	debug: Qjob submitted to QPU 2.
	debug: A QuantumCircuit was provided.
	debug: Translating to dict for AerSimulator...
	debug: QJob created.
	debug:  {"config":{"shots": 100000, "method": "statevector", "memory_slots": 6, "seed": 188}, "instructions":[{"name": "ry", "qubits": [0], "params": [0.0]}, {"name": "rz", "qubits": [0], "params": [0.0]}, {"name": "ry", "qubits": [1], "params": [0.0]}, {"name": "rz", "qubits": [1], "params": [0.0]}, {"name": "ry", "qubits": [2], "params": [0.0]}, {"name": "rz", "qubits": [2], "params": [0.0]}, {"name": "ry", "qubits": [3], "params": [0.0]}, {"name": "rz", "qubits": [3], "params": [0.0]}, {"name": "ry", "qubits": [4], "params": [0.0]}, {"name": "rz", "qubits": [4], "params": [0.0]}, {"name": "ry", "qubits": [5], "params": [0.0]}, {"name": "rz", "qubits": [5], "params": [0.0]}, {"name": "cx", "qubits": [0, 1], "params": []}, {"name": "cx", "qubits": [1, 2], "params": []}, {"name": "cx", "qubits": [2, 3], "params": []}, {

	debug: Circuit was sent.
	debug: Qjob submitted to QPU 5.
	debug: A QuantumCircuit was provided.
	debug: Translating to dict for AerSimulator...
	debug: QJob created.
	debug:  {"config":{"shots": 100000, "method": "statevector", "memory_slots": 6, "seed": 188}, "instructions":[{"name": "ry", "qubits": [0], "params": [0.0]}, {"name": "rz", "qubits": [0], "params": [0.0]}, {"name": "ry", "qubits": [1], "params": [0.0]}, {"name": "rz", "qubits": [1], "params": [0.0]}, {"name": "ry", "qubits": [2], "params": [0.0]}, {"name": "rz", "qubits": [2], "params": [0.0]}, {"name": "ry", "qubits": [3], "params": [0.0]}, {"name": "rz", "qubits": [3], "params": [0.0]}, {"name": "ry", "qubits": [4], "params": [0.0]}, {"name": "rz", "qubits": [4], "params": [0.0]}, {"name": "ry", "qubits": [5], "params": [0.0]}, {"name": "rz", "qubits": [5], "params": [0.0]}, {"name": "cx", "qubits": [0, 1], "params": []}, {"name": "cx", "qubits": [1, 2], "params": []}, {"name": "cx", "qubits": [2, 3], "params": []}, {

	debug: Circuit was sent.
	debug: Qjob submitted to QPU 8.
	debug: A QuantumCircuit was provided.
	debug: Translating to dict for AerSimulator...
	debug: QJob created.
	debug:  {"config":{"shots": 100000, "method": "statevector", "memory_slots": 6, "seed": 188}, "instructions":[{"name": "ry", "qubits": [0], "params": [0.0]}, {"name": "rz", "qubits": [0], "params": [0.0]}, {"name": "ry", "qubits": [1], "params": [0.0]}, {"name": "rz", "qubits": [1], "params": [0.0]}, {"name": "ry", "qubits": [2], "params": [0.0]}, {"name": "rz", "qubits": [2], "params": [0.0]}, {"name": "ry", "qubits": [3], "params": [0.0]}, {"name": "rz", "qubits": [3], "params": [0.0]}, {"name": "ry", "qubits": [4], "params": [0.0]}, {"name": "rz", "qubits": [4], "params": [0.0]}, {"name": "ry", "qubits": [5], "params": [0.0]}, {"name": "rz", "qubits": [5], "params": [0.0]}, {"name": "cx", "qubits": [0, 1], "params": []}, {"name": "cx", "qubits": [1, 2], "params": []}, {"name": "cx", "qubits": [2, 3], "params": []}, {

	debug: Circuit was sent.
	debug: Qjob submitted to QPU 11.
	debug: A QuantumCircuit was provided.
	debug: Translating to dict for AerSimulator...
	debug: QJob created.
	debug:  {"config":{"shots": 100000, "method": "statevector", "memory_slots": 6, "seed": 188}, "instructions":[{"name": "ry", "qubits": [0], "params": [0.0]}, {"name": "rz", "qubits": [0], "params": [0.0]}, {"name": "ry", "qubits": [1], "params": [0.0]}, {"name": "rz", "qubits": [1], "params": [0.0]}, {"name": "ry", "qubits": [2], "params": [0.0]}, {"name": "rz", "qubits": [2], "params": [0.0]}, {"name": "ry", "qubits": [3], "params": [0.0]}, {"name": "rz", "qubits": [3], "params": [0.0]}, {"name": "ry", "qubits": [4], "params": [0.0]}, {"name": "rz", "qubits": [4], "params": [0.0]}, {"name": "ry", "qubits": [5], "params": [0.0]}, {"name": "rz", "qubits": [5], "params": [0.0]}, {"name": "cx", "qubits": [0, 1], "params": []}, {"name": "cx", "qubits": [1, 2], "params": []}, {"name": "cx", "qubits": [2, 3], "params": []}, 

...

	debug: Circuit was sent.
	debug: Qjob submitted to QPU 17.
	debug: A QuantumCircuit was provided.
	debug: Translating to dict for AerSimulator...
	debug: QJob created.
	debug:  {"config":{"shots": 100000, "method": "statevector", "memory_slots": 6, "seed": 188}, "instructions":[{"name": "ry", "qubits": [0], "params": [0.0]}, {"name": "rz", "qubits": [0], "params": [0.0]}, {"name": "ry", "qubits": [1], "params": [0.0]}, {"name": "rz", "qubits": [1], "params": [0.0]}, {"name": "ry", "qubits": [2], "params": [0.0]}, {"name": "rz", "qubits": [2], "params": [0.0]}, {"name": "ry", "qubits": [3], "params": [0.0]}, {"name": "rz", "qubits": [3], "params": [0.0]}, {"name": "ry", "qubits": [4], "params": [0.0]}, {"name": "rz", "qubits": [4], "params": [0.0]}, {"name": "ry", "qubits": [5], "params": [0.0]}, {"name": "rz", "qubits": [5], "params": [0.0]}, {"name": "cx", "qubits": [0, 1], "params": []}, {"name": "cx", "qubits": [1, 2], "params": []}, {"name": "cx", "qubits": [2, 3], "params": []}, 

	debug: Circuit was sent.
	debug: Qjob submitted to QPU 20.
	debug: A QuantumCircuit was provided.
	debug: Translating to dict for AerSimulator...
	debug: QJob created.
	debug:  {"config":{"shots": 100000, "method": "statevector", "memory_slots": 6, "seed": 188}, "instructions":[{"name": "ry", "qubits": [0], "params": [0.0]}, {"name": "rz", "qubits": [0], "params": [0.0]}, {"name": "ry", "qubits": [1], "params": [0.0]}, {"name": "rz", "qubits": [1], "params": [0.0]}, {"name": "ry", "qubits": [2], "params": [0.0]}, {"name": "rz", "qubits": [2], "params": [0.0]}, {"name": "ry", "qubits": [3], "params": [0.0]}, {"name": "rz", "qubits": [3], "params": [0.0]}, {"name": "ry", "qubits": [4], "params": [0.0]}, {"name": "rz", "qubits": [4], "params": [0.0]}, {"name": "ry", "qubits": [5], "params": [0.0]}, {"name": "rz", "qubits": [5], "params": [0.0]}, {"name": "cx", "qubits": [0, 1], "params": []}, {"name": "cx", "qubits": [1, 2], "params": []}, {"name": "cx", "qubits": [2, 3], "params": []}, 

	debug: Circuit was sent.
	debug: Qjob submitted to QPU 23.
	debug: A QuantumCircuit was provided.
	debug: Translating to dict for AerSimulator...
	debug: QJob created.
	debug:  {"config":{"shots": 100000, "method": "statevector", "memory_slots": 6, "seed": 188}, "instructions":[{"name": "ry", "qubits": [0], "params": [0.0]}, {"name": "rz", "qubits": [0], "params": [0.0]}, {"name": "ry", "qubits": [1], "params": [0.0]}, {"name": "rz", "qubits": [1], "params": [0.0]}, {"name": "ry", "qubits": [2], "params": [0.0]}, {"name": "rz", "qubits": [2], "params": [0.0]}, {"name": "ry", "qubits": [3], "params": [0.0]}, {"name": "rz", "qubits": [3], "params": [0.0]}, {"name": "ry", "qubits": [4], "params": [0.0]}, {"name": "rz", "qubits": [4], "params": [0.0]}, {"name": "ry", "qubits": [5], "params": [0.0]}, {"name": "rz", "qubits": [5], "params": [0.0]}, {"name": "cx", "qubits": [0, 1], "params": []}, {"name": "cx", "qubits": [1, 2], "params": []}, {"name": "cx", "qubits": [2, 3], "params": []}, 

	debug: Circuit was sent.
	debug: Qjob submitted to QPU 26.
	debug: A QuantumCircuit was provided.
	debug: Translating to dict for AerSimulator...
	debug: QJob created.
	debug:  {"config":{"shots": 100000, "method": "statevector", "memory_slots": 6, "seed": 188}, "instructions":[{"name": "ry", "qubits": [0], "params": [0.0]}, {"name": "rz", "qubits": [0], "params": [0.0]}, {"name": "ry", "qubits": [1], "params": [0.0]}, {"name": "rz", "qubits": [1], "params": [0.0]}, {"name": "ry", "qubits": [2], "params": [0.0]}, {"name": "rz", "qubits": [2], "params": [0.0]}, {"name": "ry", "qubits": [3], "params": [0.0]}, {"name": "rz", "qubits": [3], "params": [0.0]}, {"name": "ry", "qubits": [4], "params": [0.0]}, {"name": "rz", "qubits": [4], "params": [0.0]}, {"name": "ry", "qubits": [5], "params": [0.0]}, {"name": "rz", "qubits": [5], "params": [0.0]}, {"name": "cx", "qubits": [0, 1], "params": []}, {"name": "cx", "qubits": [1, 2], "params": []}, {"name": "cx", "qubits": [2, 3], "params": []}, 

	debug: Circuit was sent.
	debug: Qjob submitted to QPU 29.
	debug: QJobMapper initialized with 30 QJobs.


In [12]:
pop=[]
total_pop=1*num_parameters
for j in range(total_pop):
    initial_point=np.random.uniform(-np.pi, np.pi, num_parameters)
    pop.append(initial_point)

bounds=[]
for i in range(0,num_parameters):
    bounds.append((-np.pi,np.pi))

print("Bounds:", len(bounds))
print("Initial population:", len(pop))

best_individual = []

def cb(xk,convergence=1e-8):
     best_individual.append(xk)

from scipy.optimize import differential_evolution
import time

tick = time.time()
result = differential_evolution(cost_function, bounds, maxiter=10, disp=True, workers=mapper, strategy='best1bin', init=pop, polish = False, callback=cb)
tack = time.time()
print(result)

energies = mapper(cost_function, best_individual)



print("Time:", tack-tick)

Bounds: 36
Initial population: 36
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x154893212580> in QPU 0...
	debug: Sending parameters to QPU 0.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x1548ccf67e20> in QPU 1...
	debug: Sending parameters to QPU 1.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x1548ccf67820> in QPU 2...
	debug: Sending parameters to QPU 2.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x154893212700> in QPU 3...
	debug: Sending parameters to QPU 3.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x1548cc0f2040> in QPU 4...
	debug: Sending parameters to QPU 4.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x15489323c280> in QPU 5...
	debug: Sending parameters to QPU 5.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x154893212670> in QPU 6...
	debug: Sending parameters to QPU 6.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x154893212490> in QPU 7...
	de

	debug: Sending parameters to QPU 11.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x15488d75ffa0> in QPU 12...
	debug: Sending parameters to QPU 12.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x15488d75fd90> in QPU 13...
	debug: Sending parameters to QPU 13.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x154893212f10> in QPU 14...
	debug: Sending parameters to QPU 14.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x15488d6ac610> in QPU 15...
	debug: Sending parameters to QPU 15.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x15488d75f3a0> in QPU 16...
	debug: Sending parameters to QPU 16.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x15488d6ac9d0> in QPU 17...
	debug: Sending parameters to QPU 17.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x15488d6acbe0> in QPU 18...
	debug: Sending parameters to QPU 18.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x15488d6ace2

/opt/cesga/qmio/hpc/software/Compiler/gcc/12.3.0/scipy/1.13.0-python-3.9.9/lib/python3.9/site-packages/scipy/optimize/_differentialevolution.py:487: UserWarning: differential_evolution: the 'workers' keyword has overridden updating='immediate' to updating='deferred'
  with DifferentialEvolutionSolver(func, bounds, args=args,


	debug: Results correctly loaded.
	debug: Results correctly loaded.
	debug: Results correctly loaded.
	debug: Results correctly loaded.
	debug: Results correctly loaded.
	debug: Results correctly loaded.
	debug: Results correctly loaded.
	debug: Results correctly loaded.
	debug: Results correctly loaded.
	debug: Results correctly loaded.
	debug: Results correctly loaded.
	debug: Results correctly loaded.
	debug: Results correctly loaded.
	debug: Results correctly loaded.
	debug: Results correctly loaded.
	debug: Results correctly loaded.
	debug: Results correctly loaded.
	debug: Results correctly loaded.
	debug: Results correctly loaded.
	debug: Results correctly loaded.
	debug: Results correctly loaded.
	debug: Results correctly loaded.
	debug: Results correctly loaded.
	debug: Results correctly loaded.
differential_evolution step 1: f(x)= 0.33932245317919957
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x154893212580> in QPU 0...
	debug: Sending parameters to QPU 0.
	d

	debug: Sending parameters to QPU 3.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x1548cc0f2040> in QPU 4...
	debug: Sending parameters to QPU 4.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x15489323c280> in QPU 5...
	debug: Sending parameters to QPU 5.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x154893212670> in QPU 6...
	debug: Sending parameters to QPU 6.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x154893212490> in QPU 7...
	debug: Sending parameters to QPU 7.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x15488d75f850> in QPU 8...
	debug: Sending parameters to QPU 8.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x15488d75fa90> in QPU 9...
	debug: Sending parameters to QPU 9.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x15488d75fcd0> in QPU 10...
	debug: Sending parameters to QPU 10.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x15488d75fdf0> in QPU 11.

...

	debug: Sending parameters to QPU 26.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x15488d6ac910> in QPU 27...
	debug: Sending parameters to QPU 27.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x15488d670910> in QPU 28...
	debug: Sending parameters to QPU 28.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x15488d670970> in QPU 29...
	debug: Sending parameters to QPU 29.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x154893212580> in QPU 0...
	debug: Sending parameters to QPU 0.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x1548ccf67e20> in QPU 1...
	debug: Sending parameters to QPU 1.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x1548ccf67820> in QPU 2...
	debug: Sending parameters to QPU 2.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x154893212700> in QPU 3...
	debug: Sending parameters to QPU 3.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x1548cc0f2040> in QP

	debug: Results correctly loaded.
	debug: Results correctly loaded.
	debug: Results correctly loaded.
	debug: Results correctly loaded.
	debug: Results correctly loaded.
	debug: Results correctly loaded.
	debug: Results correctly loaded.
	debug: Results correctly loaded.
	debug: Results correctly loaded.
	debug: Results correctly loaded.
	debug: Results correctly loaded.
	debug: Results correctly loaded.
	debug: Results correctly loaded.
	debug: Results correctly loaded.
	debug: Results correctly loaded.
	debug: Results correctly loaded.
	debug: Results correctly loaded.
	debug: Results correctly loaded.
	debug: Results correctly loaded.
	debug: Results correctly loaded.
	debug: Results correctly loaded.
	debug: Results correctly loaded.
	debug: Results correctly loaded.
	debug: Results correctly loaded.
differential_evolution step 6: f(x)= 0.3118018307998417
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x154893212580> in QPU 0...
	debug: Sending parameters to QPU 0.
	de

	debug: Sending parameters to QPU 3.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x1548cc0f2040> in QPU 4...
	debug: Sending parameters to QPU 4.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x15489323c280> in QPU 5...
	debug: Sending parameters to QPU 5.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x154893212670> in QPU 6...
	debug: Sending parameters to QPU 6.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x154893212490> in QPU 7...
	debug: Sending parameters to QPU 7.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x15488d75f850> in QPU 8...
	debug: Sending parameters to QPU 8.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x15488d75fa90> in QPU 9...
	debug: Sending parameters to QPU 9.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x15488d75fcd0> in QPU 10...
	debug: Sending parameters to QPU 10.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x15488d75fdf0> in QPU 11.

	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x15488d6ac610> in QPU 15...
	debug: Sending parameters to QPU 15.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x15488d75f3a0> in QPU 16...
	debug: Sending parameters to QPU 16.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x15488d6ac9d0> in QPU 17...
	debug: Sending parameters to QPU 17.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x15488d6acbe0> in QPU 18...
	debug: Sending parameters to QPU 18.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x15488d6ace20> in QPU 19...
	debug: Sending parameters to QPU 19.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x15488d6acf70> in QPU 20...
	debug: Sending parameters to QPU 20.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x15488d6aca90> in QPU 21...
	debug: Sending parameters to QPU 21.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x15488d6acf40> in QPU 22...
	debug: Sending parame

	debug: Sending parameters to QPU 26.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x15488d6ac910> in QPU 27...
	debug: Sending parameters to QPU 27.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x15488d670910> in QPU 28...
	debug: Sending parameters to QPU 28.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x15488d670970> in QPU 29...
	debug: Sending parameters to QPU 29.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x154893212580> in QPU 0...
	debug: Sending parameters to QPU 0.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x1548ccf67e20> in QPU 1...
	debug: Sending parameters to QPU 1.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x1548ccf67820> in QPU 2...
	debug: Sending parameters to QPU 2.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x154893212700> in QPU 3...
	debug: Sending parameters to QPU 3.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x1548cc0f2040> in QP

In [13]:
dir(result)

['fun',
 'message',
 'nfev',
 'nit',
 'population',
 'population_energies',
 'success',
 'x']

In [14]:
result.population_energies

array([0.31180183, 0.44519048, 0.57301629, 0.55244363, 0.45162242,
       0.43940695, 0.3484831 , 0.43494835, 0.49818026, 0.43753745,
       0.47745906, 0.49490684, 0.33932245, 0.59890513, 0.46348694,
       0.43559463, 0.50558041, 0.44430924, 0.41752788, 0.47820639,
       0.51162346, 0.50484704, 0.45938112, 0.37113412, 0.36354123,
       0.41141311, 0.54164276, 0.48954013, 0.52431021, 0.42279866,
       0.33932245, 0.44519048, 0.57301629, 0.55244363, 0.45162242,
       0.43940695])

In [16]:
result.x

array([-2.02434117, -2.00047377, -2.80355351,  2.45903335,  2.18912691,
       -1.6116901 , -0.75460633,  3.02696771,  1.29349519,  0.95809387,
        0.53732666, -0.21149069, -1.17646884, -1.85891381, -2.18614445,
       -0.44702746, -1.66061301, -1.75446131,  2.01189409, -0.81606768,
       -0.24320534,  2.08751171, -1.16819126,  2.76984947,  2.68651542,
       -2.20850762, -2.90882978, -0.99749139,  0.49409542,  0.44532877,
        2.42858615,  1.85962271, -1.80884367,  1.92712064, -1.56510203,
       -1.95484304])

In [17]:
circ = ansatz.assign_parameters(result.x)

In [33]:
energies

[0.727562237626633,
 0.5375982776510955,
 0.8481303502149584,
 0.7380450645063268,
 0.7884340047902858,
 0.6218309345052601,
 0.9141965358010642,
 0.9069933133442408,
 0.9017151546155266,
 0.908658796430865,
 0.9044093999901259,
 0.9115332860362739,
 0.9047464179405276,
 0.9039963032741348,
 0.9050659607038982,
 0.9065477748138756,
 0.9043032477440331,
 0.8997411709938532,
 0.9063042647608612,
 0.9087510880924539,
 0.9028235939702883,
 0.9028993728877099,
 0.9086315120488707,
 0.9060376759219009,
 0.9052891321963904,
 0.909549649983757,
 0.9078178329831255,
 0.9078002052833902,
 0.23625076793234898,
 0.23627627817923913,
 0.727562237626633,
 0.5375982776510955,
 0.8481303502149584,
 0.7380450645063268,
 0.7884340047902858,
 0.6218309345052601,
 0.9141965358010642,
 0.9069933133442408,
 0.9017151546155266,
 0.908658796430865,
 0.9044093999901259,
 0.9115332860362739,
 0.9047464179405276,
 0.9039963032741348,
 0.9050659607038982,
 0.9065477748138756,
 0.9043032477440331,
 0.8997411709938

In [22]:
for i in range(10):
    print(qpus[0].backend.name)
    counts = qpus[0].run(circ, shots = n_shots, seed_simulator = 12).result().seed_simulator
    print(counts)

BasicAer
	debug: A QuantumCircuit was provided.
	debug: Translating to dict for AerSimulator...
	debug: QJob created.
	debug:  {"config":{"shots": 100000, "method": "statevector", "memory_slots": 6, "seed": 188, "seed_simulator": 12}, "instructions":[{"name": "ry", "qubits": [0], "params": [2.011894086375032]}, {"name": "rz", "qubits": [0], "params": [-2.0243411663210233]}, {"name": "ry", "qubits": [1], "params": [-0.8160676804917514]}, {"name": "rz", "qubits": [1], "params": [-2.000473766536703]}, {"name": "ry", "qubits": [2], "params": [-0.24320534434937585]}, {"name": "rz", "qubits": [2], "params": [-2.803553510163279]}, {"name": "ry", "qubits": [3], "params": [2.087511710996042]}, {"name": "rz", "qubits": [3], "params": [2.4590333520799437]}, {"name": "ry", "qubits": [4], "params": [-1.1681912574664184]}, {"name": "rz", "qubits": [4], "params": [2.1891269115422367]}, {"name": "ry", "qubits": [5], "params": [2.7698494674294776]}, {"name": "rz", "qubits": [5], "params": [-1.611690097

	debug: Circuit was sent.
	debug: Qjob submitted to QPU 0.
	debug: Results correctly loaded.
804328122
BasicAer
	debug: A QuantumCircuit was provided.
	debug: Translating to dict for AerSimulator...
	debug: QJob created.
	debug:  {"config":{"shots": 100000, "method": "statevector", "memory_slots": 6, "seed": 188, "seed_simulator": 12}, "instructions":[{"name": "ry", "qubits": [0], "params": [2.011894086375032]}, {"name": "rz", "qubits": [0], "params": [-2.0243411663210233]}, {"name": "ry", "qubits": [1], "params": [-0.8160676804917514]}, {"name": "rz", "qubits": [1], "params": [-2.000473766536703]}, {"name": "ry", "qubits": [2], "params": [-0.24320534434937585]}, {"name": "rz", "qubits": [2], "params": [-2.803553510163279]}, {"name": "ry", "qubits": [3], "params": [2.087511710996042]}, {"name": "rz", "qubits": [3], "params": [2.4590333520799437]}, {"name": "ry", "qubits": [4], "params": [-1.1681912574664184]}, {"name": "rz", "qubits": [4], "params": [2.1891269115422367]}, {"name": "ry"

	debug: Circuit was sent.
	debug: Qjob submitted to QPU 0.
	debug: Results correctly loaded.
3667339429
BasicAer
	debug: A QuantumCircuit was provided.
	debug: Translating to dict for AerSimulator...
	debug: QJob created.
	debug:  {"config":{"shots": 100000, "method": "statevector", "memory_slots": 6, "seed": 188, "seed_simulator": 12}, "instructions":[{"name": "ry", "qubits": [0], "params": [2.011894086375032]}, {"name": "rz", "qubits": [0], "params": [-2.0243411663210233]}, {"name": "ry", "qubits": [1], "params": [-0.8160676804917514]}, {"name": "rz", "qubits": [1], "params": [-2.000473766536703]}, {"name": "ry", "qubits": [2], "params": [-0.24320534434937585]}, {"name": "rz", "qubits": [2], "params": [-2.803553510163279]}, {"name": "ry", "qubits": [3], "params": [2.087511710996042]}, {"name": "rz", "qubits": [3], "params": [2.4590333520799437]}, {"name": "ry", "qubits": [4], "params": [-1.1681912574664184]}, {"name": "rz", "qubits": [4], "params": [2.1891269115422367]}, {"name": "ry

	debug: Circuit was sent.
	debug: Qjob submitted to QPU 0.
	debug: Results correctly loaded.
3792004311
BasicAer
	debug: A QuantumCircuit was provided.
	debug: Translating to dict for AerSimulator...
	debug: QJob created.
	debug:  {"config":{"shots": 100000, "method": "statevector", "memory_slots": 6, "seed": 188, "seed_simulator": 12}, "instructions":[{"name": "ry", "qubits": [0], "params": [2.011894086375032]}, {"name": "rz", "qubits": [0], "params": [-2.0243411663210233]}, {"name": "ry", "qubits": [1], "params": [-0.8160676804917514]}, {"name": "rz", "qubits": [1], "params": [-2.000473766536703]}, {"name": "ry", "qubits": [2], "params": [-0.24320534434937585]}, {"name": "rz", "qubits": [2], "params": [-2.803553510163279]}, {"name": "ry", "qubits": [3], "params": [2.087511710996042]}, {"name": "rz", "qubits": [3], "params": [2.4590333520799437]}, {"name": "ry", "qubits": [4], "params": [-1.1681912574664184]}, {"name": "rz", "qubits": [4], "params": [2.1891269115422367]}, {"name": "ry

In [22]:
from qiskit import QuantumCircuit
qc = QuantumCircuit(2)
qc.rx(1.77, 0)
qc.measure_all()

for i in range(10):
    print(qpus[0].backend.name)
    counts = qpus[0].run(qc, shots = n_shots, seed = 8).result().get_counts()
    print(counts)

BasicAer
{'00': 99, '01': 1034, '10': 1039, '11': 826}
BasicAer
{'00': 73, '01': 996, '10': 1024, '11': 821}
BasicAer
{'00': 104, '01': 1001, '10': 958, '11': 868}
BasicAer
{'00': 82, '01': 1056, '10': 970, '11': 742}
BasicAer
{'00': 79, '01': 1066, '10': 1001, '11': 796}
BasicAer
{'00': 93, '01': 1026, '10': 977, '11': 768}
BasicAer
{'00': 97, '01': 989, '10': 985, '11': 893}
BasicAer
{'00': 83, '01': 1004, '10': 973, '11': 801}
BasicAer
{'00': 87, '01': 975, '10': 930, '11': 848}
BasicAer
{'00': 79, '01': 988, '10': 1028, '11': 847}


In [18]:
import matplotlib.pyplot as plt
plt.clf()
plt.plot(np.linspace(0, result.nit, result.nit), energies, label="Optimization path (run())")
upper_bound = result.nit
plt.plot(np.linspace(0, upper_bound, upper_bound), np.zeros(upper_bound), "--", label="Target cost")
plt.xlabel("Step"); plt.ylabel("Cost"); plt.legend(loc="upper right"); plt.title(f"n = {num_qubits}, l = {num_layers}, # params = {num_parameters}")
plt.grid(True)
plt.show()
plt.savefig(f"optimization_de_n_{num_qubits}_p_{num_parameters}.png", dpi=200)

In [19]:
lista = [-1.8717609475089807, 2.919760044838526, -1.4124298801537238, 0.9161765699704417, 2.562883105085934, 0.5667637571988291, -2.5725059694192596, 1.235824416778379, -2.9138794541998276, 1.6527400030504462, -2.3482597419936413, 1.9993961466576764, 1.5566463178936045, 0.12987821063892105, 1.0363964446556453, -0.032835304283010125, 0.7800599218456872, 0.6136567977176869, -2.388218620053299, 0.3381876004849084, 2.5001076969431364, 2.498723705381258, 1.158219355120325, -3.140265916264401, -0.6061889006504347, -1.7942180414387496, 2.963679771582688, 2.465252827878361, -2.344759126455823, -1.00580332790408, 1.9092771321163728, -1.6788882738971813, 1.6132266184995945, -1.1472345524117762, -2.119135658127555, 0.5194481651687578]

In [22]:
len([lista])

1

# Paralelization of expectation value terms

In [25]:
# TODO

# Paralelization for gradient optimizers

In [26]:
# TODO